In [14]:
import numpy as np
import pandas
from sklearn.cross_validation import train_test_split

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#1. Загрузите выборку из файла gbm-data.csv с помощью pandas и преобразуйте ее в массив numpy 
# (параметр values у датафрейма). В первой колонке файла с данными записано, была или нет реакция. 


In [4]:
data = pandas.read_csv('gbm-data.csv')
data[:2]

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0


In [11]:
y = data.loc[:,'Activity'].values
y[:5]

array([1, 1, 1, 1, 0])

In [ ]:
# Все остальные колонки (d1 - d1776) содержат различные характеристики молекулы, такие как размер, форма и т.д. 

In [12]:
X = data.loc[:,'D1':'D1776'].values
X[:2]

array([[0.        , 0.49700901, 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.36666667, 0.60629148, 0.05      , ..., 0.        , 1.        ,
        0.        ]])

In [15]:
# Разбейте выборку на обучающую и тестовую, используя функцию train_test_split с параметрами test_size = 0.8 
#и random_state = 241.
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.8, random_state=241)

In [ ]:
# 2. Обучите GradientBoostingClassifier с параметрами n_estimators=250, verbose=True, random_state=241 и 
# для каждого значения learning_rate из списка [1, 0.5, 0.3, 0.2, 0.1] проделайте следующее: